In [1]:
# --- STEP 1: SETUP & MERGE DATASETS ---
!pip install ultralytics -q
!pip install opencv-python pycocotools matplotlib

# --- CORE IMPORTS ---
import os, shutil, io, random, cv2, numpy as np, pandas as pd
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt


# --- Torchvision ---
from torchvision import transforms, datasets, models
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# --- For K-Fold ---
from sklearn.model_selection import StratifiedKFold
print("✅ All imports done")

from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score

import torch.optim as optim
from torch.nn import CrossEntropyLoss


#Multiprocessing
!pip install mpire
import mpire as _noop  # no-op import to keep dependencies explicit (optional)
import multiprocessing as mp
import tqdm

#dataset creation
import json
import subprocess, sys, glob

# --- YOLO ---
from ultralytics import YOLO

#-----------------------------------------------------
print("Done")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00:00:0100:01
✅ All imports done


In [2]:
import os
import shutil
import pandas as pd

# Dataset paths
bovine_dataset = '/kaggle/input/indian-bovine-breeds/'
calf_dataset   = '/kaggle/input/indian-cow-calf-images/'

# Check existence
assert os.path.exists(bovine_dataset), "Bovine dataset not found!"
assert os.path.exists(calf_dataset), "Calf dataset not found!"

# Merged dataset base
merged_base = '/kaggle/working/merged_dataset'
csv_path = os.path.join(merged_base, "labels.csv")

# Preprocessed dataset already exists
if os.path.exists(merged_base) and os.path.exists(csv_path):
    print("✅ Preprocessed dataset found, loading CSV...")
    df = pd.read_csv(csv_path)
else:
    if os.path.exists(merged_base):
        shutil.rmtree(merged_base)
    os.makedirs(merged_base, exist_ok=True)
    print("⚙ Dataset will be preprocessed for all breeds (including calves).")
    
    all_rows = []

    # Adult bovine images
    for root, dirs, files in os.walk(bovine_dataset):
        breed = os.path.basename(root)
        if breed == os.path.basename(os.path.normpath(bovine_dataset)):
            continue
        for img in sorted(files):
            if img.lower().endswith(('.jpg','.jpeg','.png')):
                all_rows.append([os.path.join(breed, img), breed, 0])

    # Calf images — ensure all images counted
    for root, dirs, files in os.walk(calf_dataset):
        for img in sorted(files):
            if img.lower().endswith(('.jpg','.jpeg','.png')):
                all_rows.append([os.path.join("calf", img), "calf", 1])

    df = pd.DataFrame(all_rows, columns=["image_path","breed","is_calf"])
    df.to_csv(csv_path, index=False)
    print(f"✅ Preprocessing complete. CSV saved at {csv_path}")

# Display number of images per breed
print("\nNumber of images per breed (including calves):")
display(df['breed'].value_counts())


⚙ Dataset will be preprocessed for all breeds (including calves).
✅ Preprocessing complete. CSV saved at /kaggle/working/merged_dataset/labels.csv

Number of images per breed (including calves):


breed
Sahiwal              439
Gir                  372
Holstein_Friesian    328
Ayrshire             234
Brown_Swiss          225
Tharparkar           217
Jersey               203
Ongole               191
Hallikar             186
Nagpuri              182
Kankrej              178
Murrah               173
Red_Dane             167
Red_Sindhi           162
Rathi                149
Vechur               140
Krishna_Valley       136
Hariana              129
Pulikulam            124
Toda                 124
Guernsey             119
Khillari             113
Banni                108
Malnad_gidda         107
Jaffrabadi           101
Alambadi              99
Deoni                 99
Kasargod              95
Amritmahal            94
Mehsana               94
Bargur                93
Kangayam              91
Nili_Ravi             88
Nagori                88
Bhadawari             86
Nimari                84
Dangi                 82
Umblachery            76
Surti                 59
Kenkatha           

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Count images per breed and sort descending
breed_counts = df['breed'].value_counts()
sorted_breeds = breed_counts.index.tolist()
sorted_counts = breed_counts.values.tolist()

# Dictionary to store checkboxes
breed_checkboxes = {}
checkbox_widgets = []

# Create checkbox for each breed
for breed, count in zip(sorted_breeds, sorted_counts):
    cb = widgets.Checkbox(
        value=True,  # default selected
        description=f"{breed} ({count} images)",
        indent=False
    )
    breed_checkboxes[breed] = cb
    checkbox_widgets.append(cb)

# Vertical box layout
breed_box = widgets.VBox(checkbox_widgets, layout=widgets.Layout(max_height='300px', overflow='auto'))
display(breed_box)

# Button to confirm selection
confirm_btn = widgets.Button(description="Confirm Selection", button_style='success')
out = widgets.Output()
display(confirm_btn, out)

# Internal variable to store selected breeds
selected_breeds_for_training = []

def on_confirm_clicked(b):
    global selected_breeds_for_training
    selected_breeds_for_training = [breed for breed, cb in breed_checkboxes.items() if cb.value]
    with out:
        clear_output()
        print(f"✅ Selected breeds for training: {selected_breeds_for_training}")

confirm_btn.on_click(on_confirm_clicked)


Button(button_style='success', description='Confirm Selection', style=ButtonStyle())

Output()

In [17]:
# Load YOLOv8 model
# You can use 'yolov8n.pt' for fast inference or your custom trained model
def load_yolo_on_device(device_id):
    device = f"cuda:{device_id}" if torch.cuda.is_available() else "cpu"
    model = YOLO('yolov8n.pt')  # replace with custom weights if available
    model.to(device)
    return model, device

print("✅ YOLO loader defined for multiple GPUs")


✅ YOLO loader defined for multiple GPUs


In [18]:
def crop_with_yolo(img_path, save_path, model, device, target_size=(300, 300)):
    """
    Detect largest cow using YOLO on a specific GPU and save cropped image.
    """
    image = cv2.imread(img_path)
    if image is None:
        print(f"⚠️ Could not read image: {img_path}")
        return False

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.predict(image_rgb, device=device, verbose=False)[0]

    boxes = results.boxes.xyxy.cpu().numpy() if len(results.boxes) > 0 else []
    if len(boxes) == 0:
        return False

    # Take the largest box (area)
    areas = [(x2-x1)*(y2-y1) for x1, y1, x2, y2 in boxes]
    largest_idx = np.argmax(areas)
    x1, y1, x2, y2 = [int(c) for c in boxes[largest_idx]]

    crop = image[y1:y2, x1:x2]
    if crop.size == 0:
        return False

    # Resize to target size
    crop_resized = cv2.resize(crop, target_size)
    Image.fromarray(cv2.cvtColor(crop_resized, cv2.COLOR_BGR2RGB)).save(save_path)
    return True
print("Done")

Done


In [19]:
#-----not needed---
def process_image(args):
    src, dst, breed, is_calf, device_id = args
    try:
        mask_generator, device = load_sam_on_device(device_id)
        if crop_with_sam(src, dst, mask_generator, device, max_dim=512):
            return [os.path.join(breed, os.path.basename(dst)), breed, is_calf]
    except Exception as e:
        print(f"⚠️ Error {src}: {e}")
    return None
print("DONE")


DONE


In [20]:


# Merged dataset base
merged_base = '/kaggle/working/merged_dataset'
csv_path = os.path.join(merged_base, "labels.csv")

os.makedirs(merged_base, exist_ok=True)
all_tasks = []

# Adult bovine images
for root, dirs, files in os.walk(bovine_dataset):
    breed = os.path.basename(root)
    if breed not in selected_breeds_for_training:
        continue
    target_dir = os.path.join(merged_base, breed)
    os.makedirs(target_dir, exist_ok=True)
    for img in sorted(files):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            src = os.path.join(root,img)
            dst = os.path.join(target_dir,img)
            all_tasks.append((src,dst,breed,0))

# Calf images
# Calf images
# Calf images
if 'calf' in selected_breeds_for_training:
    calf_target = os.path.join(merged_base,'calf')
    os.makedirs(calf_target, exist_ok=True)
    for img in sorted(os.listdir(calf_dataset)):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            src = os.path.join(calf_dataset,img)
            dst = os.path.join(calf_target,img)
            all_tasks.append((src,dst,'calf',1))


print(f"Total images to process: {len(all_tasks)}")

# Distribute tasks across GPUs
gpu_count = torch.cuda.device_count()
if gpu_count == 0:
    raise RuntimeError("No CUDA GPUs found.")
tasks_per_gpu = {i:[] for i in range(gpu_count)}
for i, task in enumerate(all_tasks):
    gpu_id = i % gpu_count
    tasks_per_gpu[gpu_id].append(task)

print(f"✅ Tasks distributed across {gpu_count} GPUs")


Total images to process: 2755
✅ Tasks distributed across 2 GPUs


In [21]:
all_rows = []

for gpu_id in range(gpu_count):
    tasks = tasks_per_gpu[gpu_id]
    if not tasks:
        print(f"GPU {gpu_id} has 0 tasks — skipping.")
        continue

    print(f"\n--- Processing {len(tasks)} images on GPU {gpu_id} ---")
    try:
        torch.cuda.set_device(gpu_id)
    except Exception:
        pass

    # Load YOLO model on this GPU
    model, device = load_yolo_on_device(gpu_id)

    pbar = tqdm.tqdm(tasks, desc=f"GPU {gpu_id}", position=gpu_id, leave=True)
    for src,dst,breed,is_calf in pbar:
        try:
            ok = crop_with_yolo(src,dst,model,device)
            if ok:
                all_rows.append([os.path.join(breed, os.path.basename(dst)), breed, is_calf])
        except Exception as e:
            print(f"⚠ Error processing {src}: {e}")

    # Free GPU memory
    try:
        del model
        torch.cuda.empty_cache()
    except Exception:
        pass

# Save CSV
if all_rows:
    df_out = pd.DataFrame(all_rows, columns=["image_path","breed","is_calf"])
    df_out.to_csv(csv_path, index=False)
    df = df_out
    print("✅ Preprocessing complete — CSV saved at:", csv_path)
    print(df['breed'].value_counts())
else:
    print("⚠ No images were successfully processed.")



--- Processing 1378 images on GPU 0 ---


GPU 0: 100%|██████████| 1378/1378 [00:50<00:00, 27.04it/s]



--- Processing 1377 images on GPU 1 ---



GPU 1:  30%|███       | 418/1377 [00:16<00:16, 57.32it/s]libpng warning: iCCP: known incorrect sRGB profile

GPU 1:  34%|███▍      | 465/1377 [00:17<00:27, 32.82it/s]libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50

GPU 1:  35%|███▌      | 486/1377 [00:18<00:27, 32.03it/s]libpng warning: iCCP: known incorrect sRGB profile

GPU 1:  89%|████████▊ | 1221/1377 [00:44<00:04, 31.42it/s]libpng warning: iCCP: known incorrect sRGB profile

GPU 1:  98%|█████████▊| 1354/1377 [00:51<00:01, 21.53it/s]libpng warning: iCCP: known incorrect sRGB profile

GPU 1: 100%|██████████| 1377/1377 [00:52<00:00, 26.41it/s]

✅ Preprocessing complete — CSV saved at: /kaggle/working/merged_dataset/labels.csv
breed
Sahiwal              427
Gir                  369
Holstein_Friesian    309
Ayrshire             222
Tharparkar           208
Brown_Swiss          206
Ongole               187
Jersey               184
Hallikar             180
Nagpuri              175
Kankrej              173
Name: count, dtype: int64


In [22]:
# Custom transform for blur, brightness, JPEG compression
class RandomImageQuality:
    def __init__(self, p_blur=0.5, p_jpeg=0.5):
        self.p_blur = p_blur
        self.p_jpeg = p_jpeg

    def __call__(self, img):
        # Random blur
        if random.random() < self.p_blur:
            img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.5, 2.0)))
        # Random JPEG compression
        if random.random() < self.p_jpeg:
            buffer = io.BytesIO()
            quality = random.randint(30, 90)
            img.save(buffer, format='JPEG', quality=quality)
            buffer.seek(0)
            img = Image.open(buffer)
        return img
print("Done")

Done


In [24]:
from PIL import Image
import random
import torchvision.transforms as transforms
import torch

class RandomOcclusion:
    def __init__(self, p=0.5, size=(20,50)):
        self.p = p          # probability of applying occlusion
        self.size = size    # min-max size of occlusion block in pixels

    def __call__(self, img):
        if random.random() < self.p:
            w, h = img.size
            # Random width and height of obstruction
            occ_w = random.randint(self.size[0], self.size[1])
            occ_h = random.randint(self.size[0], self.size[1])
            # Random top-left position
            x1 = random.randint(0, w - occ_w)
            y1 = random.randint(0, h - occ_h)
            # Draw black rectangle as obstruction
            img_pil = img.copy()
            from PIL import ImageDraw
            draw = ImageDraw.Draw(img_pil)
            draw.rectangle([x1, y1, x1+occ_w, y1+occ_h], fill=(0,0,0))
            img = img_pil
        return img
print("Done")

Done


In [25]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.Resize((300, 300)),  # Resize PIL image

    transforms.RandomApply([RandomImageQuality()], p=0.2),
    transforms.RandomApply([transforms.ColorJitter(
        brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05
    )], p=0.3),
    transforms.RandomApply([transforms.RandomRotation(degrees=30)], p=0.3),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.3),
    RandomOcclusion(p=0.3, size=(20,50)),

    transforms.ToTensor(),  # Convert PIL -> Tensor

    #transforms.RandomErasing(p=0.2, scale=(0.02,0.2), ratio=(0.3,3.3)),  # Tensor only
])

val_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor()
])

print("✅ Transforms ready")


✅ Transforms ready


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)




# --- Assign folds for K-Fold cross-validation ---
K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
df["fold"] = -1

for fold, (train_idx, val_idx) in enumerate(skf.split(df["image_path"], df["breed"])):
    df.loc[val_idx, "fold"] = fold

# Save fold info
df.to_csv(csv_path, index=False)
print(f"✅ Assigned {K} folds for cross-validation")
print("Done")

✅ Using device: cuda
✅ Assigned 5 folds for cross-validation
Done


In [29]:
class FoldDataset(Dataset):
    """
    Custom dataset for K-Fold training.
    """
    def __init__(self, df, fold, train=True, transform=None):
        self.transform = transform
        self.classes = sorted(df["breed"].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.data = df[df["fold"] != fold] if train else df[df["fold"] == fold]
        self.data = self.data.reset_index(drop=True)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(merged_base, row["image_path"])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.class_to_idx[row["breed"]]
        return img, label
print("Done")

Done


In [30]:
val_fold = 0  

# Training set = all folds except 0
train_df = df[df["fold"] != val_fold].reset_index(drop=True)

# Validation set = only fold 0
val_df   = df[df["fold"] == val_fold].reset_index(drop=True)

train_dataset = FoldDataset(train_df, fold=val_fold, train=True, transform=train_transforms)
val_dataset   = FoldDataset(val_df, fold=val_fold, train=False, transform=val_transforms)
print("Done")


Done


In [31]:
# --- STEP 10: SETUP TRAINING ---


batch_size = 16


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)



print("✅ Training setup complete")
print("Done")

✅ Training setup complete
Done


In [32]:
best_val_acc = 0.0  # track best validation accuracy

# --- Load pretrained EfficientNet-B3 ---
model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Replace classifier to match number of cow breeds
num_classes = len(df["breed"].unique())
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# --- Criterion ---
criterion = nn.CrossEntropyLoss()

# --- Gradual unfreeze schedule ---
# Each tuple: (phase_name, list of layers to unfreeze, learning_rate, epochs)
phases = [
    ("Phase 1: Train classifier only", [model.classifier], 1e-3, 20),
    ("Phase 2: Unfreeze last block (features[7])", [model.features[7]], 1e-4, 10),
    ("Phase 3: Unfreeze second-to-last block (features[6])", [model.features[6]], 1e-5, 5),
    ("Phase 4: Unfreeze third-to-last block (features[5])", [model.features[5]], 5e-6, 5)
]

best_val_loss = float('inf')
best_model_path = "/kaggle/working/best_model.pth"

# --- Training loop with gradual unfreezing ---
for phase_name, layers_to_unfreeze, lr, epochs in phases:
    # Unfreeze specified layers
    for layer in layers_to_unfreeze:
        for param in layer.parameters():
            param.requires_grad = True

    # Optimizer for trainable parameters
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Scheduler reduces LR if val_loss plateaus
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

    print(f"\n=== {phase_name} ===")

    for epoch in range(epochs):
        # --- Training ---
        model.train()
        train_loss = 0
        all_train_labels = []
        all_train_preds = []

        for images, labels in train_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            all_train_labels.extend(labels.cpu().numpy())
            all_train_preds.extend(preds.cpu().numpy())

        train_loss /= len(train_loader.dataset)
        train_acc = accuracy_score(all_train_labels, all_train_preds)
        train_f1 = f1_score(all_train_labels, all_train_preds, average='weighted')

        # --- Validation ---
        model.eval()
        val_loss = 0
        all_val_labels = []
        all_val_preds = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                preds = outputs.argmax(dim=1)
                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_acc = accuracy_score(all_val_labels, all_val_preds)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')

        # Step scheduler
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"✅ Best model updated (Val Acc: {best_val_acc:.4f})")


print(f"\n🏆 Training complete. Best model saved at {best_model_path}")
print("Done")


=== Phase 1: Train classifier only ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 | Train Loss: 1.8063, Acc: 0.4171, F1: 0.3984 | Val Loss: 1.3422, Acc: 0.6326, F1: 0.6221
✅ Best model updated (Val Acc: 0.6326)
Epoch 2/20 | Train Loss: 1.2898, Acc: 0.5994, F1: 0.5935 | Val Loss: 1.0951, Acc: 0.6913, F1: 0.6902
✅ Best model updated (Val Acc: 0.6913)
Epoch 3/20 | Train Loss: 1.1193, Acc: 0.6539, F1: 0.6500 | Val Loss: 4.2971, Acc: 0.7064, F1: 0.7068
✅ Best model updated (Val Acc: 0.7064)
Epoch 4/20 | Train Loss: 1.0553, Acc: 0.6667, F1: 0.6637 | Val Loss: 3.1902, Acc: 0.7178, F1: 0.7186
✅ Best model updated (Val Acc: 0.7178)
Epoch 5/20 | Train Loss: 1.0034, Acc: 0.6742, F1: 0.6716 | Val Loss: 1.3333, Acc: 0.7216, F1: 0.7178
✅ Best model updated (Val Acc: 0.7216)
Epoch 6/20 | Train Loss: 0.9423, Acc: 0.7055, F1: 0.7034 | Val Loss: 1.2641, Acc: 0.7292, F1: 0.7284
✅ Best model updated (Val Acc: 0.7292)
Epoch 7/20 | Train Loss: 0.9407, Acc: 0.6941, F1: 0.6924 | Val Loss: 1.2415, Acc: 0.7367, F1: 0.7374
✅ Best model updated (Val Acc: 0.7367)
Epoch 8/20 | Train L

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 | Train Loss: 0.8085, Acc: 0.7296, F1: 0.7285 | Val Loss: 1.4728, Acc: 0.7708, F1: 0.7694
✅ Best model updated (Val Acc: 0.7708)
Epoch 2/10 | Train Loss: 0.7127, Acc: 0.7704, F1: 0.7693 | Val Loss: 14.3140, Acc: 0.7614, F1: 0.7622
Epoch 3/10 | Train Loss: 0.6587, Acc: 0.7850, F1: 0.7843 | Val Loss: 2.9162, Acc: 0.7765, F1: 0.7755
✅ Best model updated (Val Acc: 0.7765)
Epoch 4/10 | Train Loss: 0.6048, Acc: 0.8054, F1: 0.8050 | Val Loss: 0.6917, Acc: 0.7898, F1: 0.7888
✅ Best model updated (Val Acc: 0.7898)
Epoch 5/10 | Train Loss: 0.5441, Acc: 0.8338, F1: 0.8330 | Val Loss: 0.7871, Acc: 0.7879, F1: 0.7879
Epoch 6/10 | Train Loss: 0.5788, Acc: 0.8120, F1: 0.8114 | Val Loss: 1.1290, Acc: 0.7917, F1: 0.7910
✅ Best model updated (Val Acc: 0.7917)
Epoch 7/10 | Train Loss: 0.4967, Acc: 0.8485, F1: 0.8481 | Val Loss: 1.9190, Acc: 0.8030, F1: 0.8023
✅ Best model updated (Val Acc: 0.8030)
Epoch 8/10 | Train Loss: 0.4448, Acc: 0.8580, F1: 0.8576 | Val Loss: 15.7956, Acc: 0.7898, F1: 0.

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5 | Train Loss: 0.4324, Acc: 0.8603, F1: 0.8600 | Val Loss: 1.1098, Acc: 0.7898, F1: 0.7883
Epoch 2/5 | Train Loss: 0.4114, Acc: 0.8693, F1: 0.8692 | Val Loss: 2.4254, Acc: 0.7879, F1: 0.7872
Epoch 3/5 | Train Loss: 0.4135, Acc: 0.8712, F1: 0.8709 | Val Loss: 1.8365, Acc: 0.8011, F1: 0.8006
Epoch 4/5 | Train Loss: 0.3745, Acc: 0.8911, F1: 0.8910 | Val Loss: 4.9293, Acc: 0.7898, F1: 0.7886
Epoch 5/5 | Train Loss: 0.3782, Acc: 0.8892, F1: 0.8890 | Val Loss: 1.2838, Acc: 0.7973, F1: 0.7961

=== Phase 4: Unfreeze third-to-last block (features[5]) ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5 | Train Loss: 0.3312, Acc: 0.9020, F1: 0.9016 | Val Loss: 1.3982, Acc: 0.8030, F1: 0.8024
Epoch 2/5 | Train Loss: 0.3612, Acc: 0.8854, F1: 0.8852 | Val Loss: 4.5401, Acc: 0.7860, F1: 0.7852
Epoch 3/5 | Train Loss: 0.3591, Acc: 0.8977, F1: 0.8974 | Val Loss: 5.9157, Acc: 0.7955, F1: 0.7946
Epoch 4/5 | Train Loss: 0.3691, Acc: 0.8883, F1: 0.8880 | Val Loss: 0.8564, Acc: 0.7973, F1: 0.7962
Epoch 5/5 | Train Loss: 0.3490, Acc: 0.8864, F1: 0.8861 | Val Loss: 2.9499, Acc: 0.7879, F1: 0.7869

🏆 Training complete. Best model saved at /kaggle/working/best_model.pth
Done


In [33]:
import torch

# Map to available device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model weights safely
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.to(device)  # move model to the device
model.eval()

print("✅ Model loaded and ready for inference on", device)


✅ Model loaded and ready for inference on cuda:0


In [34]:
# ---------------- Prepare export folder ----------------
import os, shutil, json, subprocess

merged_base = '/kaggle/working/merged_dataset'
best_model_path = '/kaggle/working/best_model.pth'
export_dir = '/kaggle/working/export_dataset'
os.makedirs(export_dir, exist_ok=True)

# Copy preprocessed dataset
if not os.path.exists(merged_base):
    raise FileNotFoundError(f"Preprocessed dataset not found at {merged_base} - run preprocessing first.")
shutil.copytree(merged_base, os.path.join(export_dir, 'merged_dataset'), dirs_exist_ok=True)

# Copy best model if exists
if os.path.exists(best_model_path):
    shutil.copy2(best_model_path, os.path.join(export_dir, os.path.basename(best_model_path)))
else:
    print("⚠️ best_model.pth not found — continuing without model.")

# Path to metadata
metadata_path = os.path.join(export_dir, 'dataset-metadata.json')

# Create minimal metadata if missing
if not os.path.exists(metadata_path):
    meta = {
        "title": "cow dataset and model 11 breeds using yolo rj",
        "id": "<username>/bovine_dataset_and_model",  # placeholder
        "licenses": [{"name": "CC0-1.0"}],
        "isPrivate": True,
        "subtitle": "SAM-cropped bovine + calf images (optional model)",
        "description": "Preprocessed dataset (Segment-Anything crops) plus optional trained model."
    }
    with open(metadata_path, 'w') as f:
        json.dump(meta, f, indent=2)
    print("✅ Created minimal dataset-metadata.json")

print("✅ Export folder prepared at:", export_dir)
print("Contents preview:", os.listdir(export_dir)[:20])
print("Done")


✅ Created minimal dataset-metadata.json
✅ Export folder prepared at: /kaggle/working/export_dataset
Contents preview: ['dataset-metadata.json', 'merged_dataset', 'best_model.pth']
Done


In [35]:
# ---------------- Kaggle CLI setup ----------------
kaggle_json_src = '/kaggle/input/kaggle-api/kaggle.json'
kaggle_local = os.path.expanduser('~/.kaggle/kaggle.json')

# Install kaggle CLI
!pip install --quiet kaggle

# Copy kaggle.json
if os.path.exists(kaggle_local):
    print("Found existing ~/.kaggle/kaggle.json — using it.")
elif os.path.exists(kaggle_json_src):
    os.makedirs(os.path.dirname(kaggle_local), exist_ok=True)
    shutil.copy2(kaggle_json_src, kaggle_local)
    os.chmod(kaggle_local, 0o600)
    print(f"Copied kaggle.json from {kaggle_json_src} -> {kaggle_local}")
else:
    raise FileNotFoundError("kaggle.json not found. Upload it via notebook UI and re-run this cell.")

# Read username
with open(kaggle_local, 'r') as f:
    kg = json.load(f)
username = kg.get('username') or kg.get('user') or kg.get('email') or None

# Update metadata id if possible
slug = 'cow-dataset-and-model-11breeds-using-yolo-rj'
if username:
    with open(metadata_path, 'r') as f:
        meta = json.load(f)
    meta['id'] = f"{username}/{slug}"
    with open(metadata_path, 'w') as f:
        json.dump(meta, f, indent=2)
    print(f"Dataset id set to: {meta['id']}")
else:
    print("⚠️ Could not determine username — dataset id in metadata will remain placeholder.")

# ---------------- Create or version Kaggle dataset ----------------
create_cmd = f"kaggle datasets create -p {export_dir} --dir-mode zip"
print("Running:", create_cmd)
res = subprocess.run(create_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if res.returncode == 0:
    print("✅ Dataset created successfully.")
    print(res.stdout)
else:
    print("Create failed; attempting to create a new version...")
    version_cmd = f"kaggle datasets version -p {export_dir} -m \"Update: SAM preproc + optional model\" --dir-mode zip --force"
    print("Running:", version_cmd)
    res2 = subprocess.run(version_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if res2.returncode == 0:
        print("✅ Dataset version created successfully.")
        print(res2.stdout)
    else:
        print("❌ Both create and version failed.")
        print("CREATE stderr:\n", res.stderr)
        print("VERSION stderr:\n", res2.stderr)
        raise RuntimeError("Failed to create/version Kaggle dataset. Check kaggle.json and metadata id.")

# ---------------- Dataset URL ----------------
if username:
    print("\nDataset should be available at:")
    print(f"https://www.kaggle.com/{username}/{slug}")
else:
    print("\nDataset created/updated but username unknown. Check Kaggle web UI.")
print("Done")


Copied kaggle.json from /kaggle/input/kaggle-api/kaggle.json -> /root/.kaggle/kaggle.json
Dataset id set to: riddhijaiswal111/cow-dataset-and-model-11breeds-using-yolo-rj
Running: kaggle datasets create -p /kaggle/working/export_dataset --dir-mode zip
✅ Dataset created successfully.
Starting upload for file merged_dataset.zip
Upload successful: merged_dataset.zip (86MB)
Starting upload for file best_model.pth
Upload successful: best_model.pth (41MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/riddhijaiswal111/cow-dataset-and-model-11breeds-using-yolo-rj


Dataset should be available at:
https://www.kaggle.com/riddhijaiswal111/cow-dataset-and-model-11breeds-using-yolo-rj
Done
